# YOLO11 Codes:

In [13]:
import torch
torch.cuda.is_available()

True

In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650'

In [3]:
import torch
torch.cuda.empty_cache()

In [5]:
!nvidia-smi

Thu Jun 26 12:04:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   49C    P8              1W /   65W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")

# Train the model
train_results = model.train(
    data="E:/project_jupyter/main_backup/data_main.yml",  # path to dataset YAML
    epochs=100,  # number of training epochs
    imgsz=640,  # training image size
    device=0,
    batch = 12,
    plots=True # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov11n.pt")

# Train the model
train_results = model.train(
    data="E:/project_jupyter/main_backup/data_main.yml",  # path to dataset YAML
    epochs=200,  # number of training epochs
    imgsz=640,  # training image size
    device=0,
    batch = 12,
    plots=True # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

In [ ]:
from ultralytics import YOLO
import cv2

# Load the trained YOLOv8 model
model = YOLO(r"C:\Users\debas\Downloads\best (2).pt")  # Replace 'best.pt' with your trained model's path if different

# Load the video
video_path = r"C:\Users\debas\Downloads\new\fighting.mp4"  # Replace with your MP4 video file
output_path = "output_videos46.mp4" # Path to save the output video
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec for MP4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference on the frame
    results = model(frame)

    # Visualize predictions on the frame
    annotated_frame = results[0].plot()

    # Write the annotated frame to the output video
    out.write(annotated_frame)

    # (Optional) Display the frame with detections in real-time
    display_frame = cv2.resize(annotated_frame, None, fx=0.5, fy=0.5)  # Adjust fx, fy as needed
    cv2.imshow("YOLO11 Inference", display_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


# FER codes:


In [20]:
import cv2
from fer import FER

# Initialize webcam and FER detector
detector = FER(mtcnn=True)  # mtcnn=True gives better face detection
cap = cv2.VideoCapture(0)   # 0 is the default webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame to speed up detection
    resized_frame = cv2.resize(frame, (640, 480))

    # Detect emotions in the frame
    results = detector.detect_emotions(resized_frame)

    # Draw boxes and labels
    for result in results:
        (x, y, w, h) = result["box"]
        emotions = result["emotions"]
        max_emotion = max(emotions, key=emotions.get)

        cv2.rectangle(resized_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(resized_frame, max_emotion, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

    # Display the frame
    cv2.imshow("FER Webcam", resized_frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch
import os
from fer import FER

# Output directory to save .pt files
output_dir = "fer_features"
os.makedirs(output_dir, exist_ok=True)

# Initialize FER detector
detector = FER(mtcnn=True)
cap = cv2.VideoCapture(0)

frame_count = 0
max_frames = 100  # Change this to capture more or fewer frames

while frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break

    resized = cv2.resize(frame, (640, 480))
    results = detector.detect_emotions(resized)

    if results:
        emotions = results[0]["emotions"]  # first face
        emotion_tensor = torch.tensor([emotions[k] for k in sorted(emotions.keys())])

        # Save tensor
        torch.save(emotion_tensor, os.path.join(output_dir, f"frame_{frame_count:04d}.pt"))

        print(f"Saved frame_{frame_count:04d}.pt --> {emotions}")

    frame_count += 1
    cv2.imshow("Capturing FER", resized)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# ✅ Settings
feature_dir = r"E:/fer_features"
csv_path = os.path.join(feature_dir, "fer_labels.csv")
batch_size = 16
epochs = 20
lr = 0.001

# ✅ Dataset
class FERDataset(Dataset):
    def __init__(self, csv_file, feature_dir, label_encoder):
        df = pd.read_csv(csv_file)
        self.samples = df.values
        self.feature_dir = feature_dir
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        filename, label = self.samples[idx]
        tensor = torch.load(os.path.join(self.feature_dir, filename))
        tensor = torch.tensor(tensor, dtype=torch.float32)

        if isinstance(label, str):
            label_idx = self.label_encoder.transform([label])[0]
        else:
            label_idx = label  # already encoded

        return tensor, label_idx



# ✅ Prepare Data
df = pd.read_csv(csv_path)
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.to_csv("train_tmp.csv", index=False)
val_df.to_csv("val_tmp.csv", index=False)

train_dataset = FERDataset("train_tmp.csv", feature_dir, le)
val_dataset = FERDataset("val_tmp.csv", feature_dir, le)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# ✅ Simple MLP Model
class FERClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(7, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

model = FERClassifier(num_classes=len(le.classes_))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# ✅ Training Loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

# ✅ Validation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for x, y in val_loader:
        out = model(x)
        preds = out.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)
print(f"Validation Accuracy: {correct / total:.2%}")

# ✅ Save model and encoder
torch.save(model.state_dict(), "fer_behavior_classifier.pt")
import pickle
with open("fer_label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

In [ ]:
import pandas as pd

# Check both CSVs
print(pd.read_csv("train_tmp.csv").head(10))
print(pd.read_csv("val_tmp.csv").head(10))

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

feature_dir = r"E:/fer_features"
csv_path = os.path.join(feature_dir, "fer_labels.csv")
batch_size = 16
epochs = 20
lr = 0.001

# Dataset
class FERDataset(Dataset):
    def __init__(self, csv_file, feature_dir):
        df = pd.read_csv(csv_file)
        self.samples = df.values
        self.feature_dir = feature_dir

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        filename, label = self.samples[idx]
        tensor = torch.load(os.path.join(self.feature_dir, filename))
        tensor = torch.tensor(tensor, dtype=torch.float32)
        label_idx = int(label)  # ✅ fixed and correctly indented
        return tensor, label_idx

# Prepare data
df = pd.read_csv(csv_path)
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.to_csv("train_tmp.csv", index=False)
val_df.to_csv("val_tmp.csv", index=False)

# Loaders
train_dataset = FERDataset("train_tmp.csv", feature_dir)
val_dataset = FERDataset("val_tmp.csv", feature_dir)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Model
class FERClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(7, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

model = FERClassifier(num_classes=len(le.classes_))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Train
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

# Validate
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for x, y in val_loader:
        out = model(x)
        preds = out.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)
print(f"Validation Accuracy: {correct / total:.2%}")

# Save model
torch.save(model.state_dict(), "fer_behavior_classifier.pt")
import pickle
with open("fer_label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


In [ ]:
import cv2
import torch
import numpy as np
import pickle
from ultralytics import YOLO
from fer import FER

# === PATHS ===
video_path = r"E:\env1gputest\our_vids\fighting_nil_deb.mp4"  # Replace with your video path
yolo_model_path = r"E:\runs\detect\train28\weights\best.pt"
fer_model_path = r"E:\fer_behavior_classifier.pt"
label_encoder_path = r"E:\fer_label_encoder.pkl"

# === LOAD MODELS ===
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load YOLOv11 model
yolo_model = YOLO("yolo11s.pt")

# Load FER feature extractor
fer_detector = FER(mtcnn=True)  # mtcnn gives better face extraction

# Load behavior classifier
class FERClassifier(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(7, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

with open(label_encoder_path, 'rb') as f:
    le = pickle.load(f)

model = FERClassifier(num_classes=len(le.classes_))
model.load_state_dict(torch.load(fer_model_path, map_location=device))
model.to(device)
model.eval()

# === INFERENCE ON VIDEO ===
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Detect person using YOLO
    results = yolo_model(frame)
    for box in results[0].boxes.data:
        cls_id = int(box[5].item())
        conf = box[4].item()
        x1, y1, x2, y2 = map(int, box[:4])

        # Skip if not person class (you can adjust if needed)
        if cls_id != 0 or conf < 0.5:
            continue

        person_crop = frame[y1:y2, x1:x2]

        # Step 2: FER Emotion extraction
        emotion_probs = fer_detector.detect_emotions(person_crop)
        if emotion_probs:
            # Grab 1st face
            emotions = emotion_probs[0]["emotions"]
            feature_vector = torch.tensor(list(emotions.values()), dtype=torch.float32).to(device)

            # Step 3: Behavior classification
            with torch.no_grad():
                output = model(feature_vector.unsqueeze(0))
                pred = output.argmax(dim=1).item()
                label = le.inverse_transform([pred])[0]

            # Step 4: Draw box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow("Behavior Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Video Classifier Using LSTM(RNN) and CNN
#!dir

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Make sure to type the path directly without any accidental hidden characters
dataset_path = os.listdir(r'E:\debs_proj\trial_dataset\train')

label_types = os.listdir(r'E:\debs_proj\trial_dataset\train')
print(label_types)

['fight', 'normal', 'theft']


# Preparing Training Data

In [2]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir(r'E:\debs_proj\trial_dataset\train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str(r'E:\debs_proj\trial_dataset\train' + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())


     tag                                    video_name
0  fight  E:\debs_proj\trial_dataset\train/fight/0.mp4
1  fight  E:\debs_proj\trial_dataset\train/fight/1.mp4
2  fight  E:\debs_proj\trial_dataset\train/fight/2.mp4
3  fight  E:\debs_proj\trial_dataset\train/fight/3.mp4
4  fight  E:\debs_proj\trial_dataset\train/fight/4.mp4
      tag                                         video_name
29  theft  E:\debs_proj\trial_dataset\train/theft/Shoplif...
30  theft  E:\debs_proj\trial_dataset\train/theft/Stealin...
31  theft  E:\debs_proj\trial_dataset\train/theft/Stealin...
32  theft  E:\debs_proj\trial_dataset\train/theft/Stealin...
33  theft  E:\debs_proj\trial_dataset\train/theft/Stealin...


In [3]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train_trial.csv')

# Preparing Test Data

In [4]:
dataset_path = os.listdir(r'E:\debs_proj\trial_dataset\test')
print(dataset_path)

room_types = os.listdir(r'E:\debs_proj\trial_dataset\test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir(r'E:\debs_proj\trial_dataset\test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str(r'E:\debs_proj\trial_dataset\test' + '/' +item) + '/' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test_trial.csv')

['fight', 'normal', 'theft']
Types of activities found:  3
      tag                                         video_name
0   fight        E:\debs_proj\trial_dataset\test/fight/0.mp4
1   fight        E:\debs_proj\trial_dataset\test/fight/1.mp4
2   fight        E:\debs_proj\trial_dataset\test/fight/2.mp4
3  normal  E:\debs_proj\trial_dataset\test/normal/Clappin...
4  normal  E:\debs_proj\trial_dataset\test/normal/Meet an...
       tag                                         video_name
8   normal  E:\debs_proj\trial_dataset\test/normal/Walking...
9   normal  E:\debs_proj\trial_dataset\test/normal/Walking...
10   theft  E:\debs_proj\trial_dataset\test/theft/rapidsav...
11   theft  E:\debs_proj\trial_dataset\test/theft/Shoplift...
12   theft  E:\debs_proj\trial_dataset\test/theft/Stealing...


In [5]:
!pip install git+https://github.com/tensorflow/docs

  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs 'C:\Users\ECE\AppData\Local\Temp\pip-req-build-ryaq4exe'

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Cloning https://github.com/tensorflow/docs to c:\users\ece\appdata\local\temp\pip-req-build-ryaq4exe
  Resolved https://github.com/tensorflow/docs to commit a8576cef38b7182e6228d7aafca8ef51754ab9e8
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


In [6]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

# Data preparation

In [8]:
train_df = pd.read_csv("train_trial.csv")
test_df = pd.read_csv("test_trial.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 34
Total videos for testing: 13


,Unnamed: 0,video_name,tag
29,29,E:\debs_proj\trial_dataset\train/theft/Shoplif...,theft
17,17,E:\debs_proj\trial_dataset\train/normal/Standi...,normal
21,21,E:\debs_proj\trial_dataset\train/normal/Walkin...,normal
28,28,E:\debs_proj\trial_dataset\train/theft/Shoplif...,theft
7,7,E:\debs_proj\trial_dataset\train/fight/Assault...,fight
10,10,E:\debs_proj\trial_dataset\train/normal/Clappi...,normal
16,16,E:\debs_proj\trial_dataset\train/normal/Sittin...,normal
19,19,E:\debs_proj\trial_dataset\train/normal/Standi...,normal
14,14,E:\debs_proj\trial_dataset\train/normal/Meet a...,normal
24,24,E:\debs_proj\trial_dataset\train/normal/Walkin...,normal


# Feed the videos to a network:


In [9]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

   ### Feature Extraction

In [10]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

### Label Encoding
StringLookup layer encode the class labels as integers.

In [11]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['fight', 'normal', 'theft']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2]], dtype=int64)

Finally, we can put all the pieces together to create our data processing utility.

In [12]:
#print(train_data[0].shape)
#train_data[0]

In [13]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 50

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [14]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
               temp_frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :], verbose=0)

            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters

Frame features in train set: (34, 20, 2048)
Frame masks in train set: (34, 20)
train_labels in train set: (34, 1)
test_labels in train set: (13, 1)


In [15]:
print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")
print(f"train_labels in train set: {train_labels.shape}")
print(f"test_labels in train set: {test_labels.shape}")


Frame features in train set: (34, 20, 2048)
Frame masks in train set: (34, 20)
train_labels in train set: (34, 1)
test_labels in train set: (13, 1)


# The sequence model
Now, we can feed this data to a sequence model consisting of recurrent layers like GRU.

In [16]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 1.1616 - accuracy: 0.4348
Epoch 1: val_loss improved from inf to 1.07378, saving model to ./tmp\video_classifier
1/1 [==============================] - 7s 7s/step - loss: 1.1616 - accuracy: 0.4348 - val_loss: 1.0738 - val_accuracy: 0.2727
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 0.9987 - accuracy: 0.4783
Epoch 2: val_loss did not improve from 1.07378
1/1 [==============================] - 0s 63ms/step - loss: 0.9987 - accuracy: 0.4783 - val_loss: 1.0992 - val_accuracy: 0.3636
Epoch 3/30
1/1 [==============================] - ETA: 0s - loss: 1.0150 - accuracy: 0.5217
Epoch 3: val_loss did not improve from 1.07378
1/1 [==============================] - 0s 62ms/step - loss: 1.0150 - accuracy: 0.5217 - val_loss: 1.1462 - val_accuracy: 0.3636
Epoch 4/30
1/1 [==============================] - ETA: 0s - loss: 0.8585 - accuracy: 0.5652
Epoch 4: val_loss did not improve from 1.07378
1/1 [=================

# Inference

In [17]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


Test video path: E:\debs_proj\trial_dataset\test/normal/Clapping (1).mp4
1/1 [==============================] - 2s 2s/step
  normal: 42.06%
  theft: 29.91%
  fight: 28.03%


In [18]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="E:/debs_proj/trial_dataset/test/fight/1.mp4" type="video/mp4">
    </video>
""")


In [19]:
video_path = "E:/debs_proj/trial_dataset/test/fight/1.mp4"
frames = load_video(video_path)


In [20]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048
video_length = frames.shape[0]
length = min(video_length, MAX_SEQ_LENGTH)

frame_mask = np.zeros((1, MAX_SEQ_LENGTH), dtype="bool")
frame_features = np.zeros((1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

for i in range(length):
    frame_features[0, i, :] = feature_extractor.predict(frames[None, i], verbose=0)

frame_mask[0, :length] = 1


In [21]:
pred = sequence_model.predict([frame_features, frame_mask])
predicted_class = np.argmax(pred, axis=1)


1/1 [==============================] - 0s 16ms/step


In [22]:
predicted_label = label_processor.get_vocabulary()[predicted_class[0]]
print(f"Predicted action: {predicted_label}")


Predicted action: normal


In [ ]:
torch.save(model.state_dict(), 'classifier.pth')


In [ ]:
git clone https://github.com/your-repo/yolo-v12
cd yolo-v12
pip install -r requirements.txt


In [ ]:
from ultralytics import YOLO

# Load a pretrained or custom trained model
yolo_model = YOLO("yolov12.pt")  # or your own trained weights


In [ ]:
import torch
import cv2
from ultralytics import YOLO
from torchvision import transforms

# Load both models
classifier = YourModelClass()
classifier.load_state_dict(torch.load('classifier.pth'))
classifier.eval()

yolo = YOLO('yolov12.pt')

# Preprocessing for classifier
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

cap = cv2.VideoCapture(r"E:\debs_proj\dataset_CNN_LSTM_method\train\fighting\2.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo(frame)

    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        roi = frame[y1:y2, x1:x2]

        # Convert to PIL and preprocess
        roi_tensor = transform(Image.fromarray(roi)).unsqueeze(0)

        # Classify the cropped ROI
        with torch.no_grad():
            output = classifier(roi_tensor)
            pred_class = output.argmax(dim=1).item()
            label = class_names[pred_class]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imshow("Result", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


# Telegram Bot Alert system

In [15]:
import requests

BOT_TOKEN = "7494981174:AAF-BEpPZaRMSnPCAnmfzHTTTHvL31Eoto0"

response = requests.get(f"https://api.telegram.org/bot{BOT_TOKEN}/getUpdates")
print(response.json())


{'ok': True, 'result': [{'update_id': 588057992, 'my_chat_member': {'chat': {'id': -1002594728756, 'title': 'final_proj', 'type': 'supergroup'}, 'from': {'id': 1389732200, 'is_bot': False, 'first_name': 'Debashish', 'last_name': 'Kashyap', 'language_code': 'en'}, 'date': 1750862758, 'old_chat_member': {'user': {'id': 7494981174, 'is_bot': True, 'first_name': 'Hi', 'username': 'Rio56664_bot'}, 'status': 'member'}, 'new_chat_member': {'user': {'id': 7494981174, 'is_bot': True, 'first_name': 'Hi', 'username': 'Rio56664_bot'}, 'status': 'administrator', 'can_be_edited': False, 'can_manage_chat': True, 'can_change_info': True, 'can_delete_messages': True, 'can_invite_users': True, 'can_restrict_members': True, 'can_pin_messages': True, 'can_manage_topics': True, 'can_promote_members': False, 'can_manage_video_chats': True, 'can_post_stories': True, 'can_edit_stories': True, 'can_delete_stories': True, 'is_anonymous': False, 'can_manage_voice_chats': True}}}]}


In [17]:
import requests

BOT_TOKEN = '7494981174:AAF-BEpPZaRMSnPCAnmfzHTTTHvL31Eoto0'
CHAT_ID = '-1002594728756'
message = "🔔 Test from Rio56664_bot"

url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
r = requests.post(url, data={"chat_id": CHAT_ID, "text": message})
print(r.json())

{'ok': True, 'result': {'message_id': 46, 'from': {'id': 7494981174, 'is_bot': True, 'first_name': 'Hi', 'username': 'Rio56664_bot'}, 'chat': {'id': -1002594728756, 'title': 'final_proj', 'type': 'supergroup'}, 'date': 1750921572, 'text': '🔔 Test from Rio56664_bot'}}


In [21]:
import cv2
import requests
from ultralytics import YOLO
import torch

# === CONFIGURATION ===
BOT_TOKEN = '7494981174:AAF-BEpPZaRMSnPCAnmfzHTTTHvL31Eoto0'
CHAT_ID = '-1002594728756'  # Group chat ID
VIDEO_PATH = r"C:\Users\debas\Downloads\CAUGHT ON CAMERA_ Austin City Hall vandalized (online-video-cutter.com).mp4"
SUSPICIOUS_CLASSES = ['theft', 'violence', 'vandalism']  # Class names to trigger alert
CONFIDENCE_THRESHOLD = 0.3

# === LOAD MODEL ===
model = YOLO(r"E:\runs\train45-20250614T153104Z-1-001\train45\weights\best.pt")  # Your trained YOLOv11s model

# === LOAD VIDEO ===
cap = cv2.VideoCapture(VIDEO_PATH)

# === SEND ALERT TO TELEGRAM ===
def send_telegram_alert(image_path, label):
    # Send text message
    message = f"⚠️ Suspicious activity detected: {label.upper()}"
    msg_url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    requests.post(msg_url, data={"chat_id": CHAT_ID, "text": message})

    # Send image
    img_url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendPhoto"
    with open(image_path, "rb") as photo:
        requests.post(img_url, data={"chat_id": CHAT_ID, "caption": "Snapshot of detected activity"}, files={"photo": photo})

# === START INFERENCE LOOP ===
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, imgsz=512)
    boxes = results[0].boxes
    classes = boxes.cls
    scores = boxes.conf

    for cls_id, conf in zip(classes, scores):
        class_name = model.names[int(cls_id)]
        if class_name in SUSPICIOUS_CLASSES and conf > CONFIDENCE_THRESHOLD:
            # Draw box for visualization (optional)
            annotated_frame = results[0].plot()

            # Save frame as image
            image_path = "alert.jpg"
            cv2.imwrite(image_path, annotated_frame)

            # Send alert
            send_telegram_alert(image_path, class_name)

            print(f"🚨 ALERT: {class_name} detected with confidence {conf:.2f}")
            break  # Alert once per suspicious frame to avoid spam

cap.release()
cv2.destroyAllWindows()



0: 288x512 1 normal, 25.5ms
Speed: 4.9ms preprocess, 25.5ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 normal, 26.3ms
Speed: 1.6ms preprocess, 26.3ms inference, 3.1ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 normal, 27.4ms
Speed: 2.1ms preprocess, 27.4ms inference, 2.7ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 normal, 16.1ms
Speed: 1.8ms preprocess, 16.1ms inference, 8.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 normal, 12.7ms
Speed: 3.5ms preprocess, 12.7ms inference, 8.1ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 normal, 23.1ms
Speed: 1.1ms preprocess, 23.1ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 normal, 25.9ms
Speed: 7.0ms preprocess, 25.9ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 normal, 17.6ms
Speed: 6.0ms preprocess, 17.6ms inference, 2.0ms postprocess per image at shape (1, 3, 28

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))